In [6]:
import os
import sys
import autogen
import pandas as pd
from typing import Dict
import pulp
import json
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define the LLM configuration
llm_config = {
    "model": "gpt-4o-mini",
    "api_key": os.environ.get("OPENAI_API_KEY"),
    "temperature": 0,
    "seed": 42,
}

# User Input Agent
user_input_agent = autogen.AssistantAgent(
    name="UserInputAgent",
    llm_config=llm_config,
    system_message="""You are an agent that processes user requests for fantasy football lineups.
    Parse the user's input and convert it into a structured format for lineup optimization.
    If the input is empty or doesn't contain specific constraints, return an empty dictionary.
    Use the following examples as a guide:

    Input: "I want Justin Jefferson in my lineup"
    Output: {"must_include": ["justinjefferson"]}

    Input: "Can you make sure to prioritize running backs?"
    Output: {"position_emphasis": {"RB": 2.0}}

    Input: "I'd like to focus on players from the Vikings"
    Output: {"team_preference": "MIN"}

    Input: "Generate me 2 unique lineups"
    Output: {"num_lineups": 2}

    Input: "Create 3 lineups with different strategies"
    Output: {"num_lineups": 3, "diverse_strategies": true}

    Input: "" (empty input)
    Output: {}

    Provide your output as a valid JSON object."""
)

# Lineup Optimizer Agent
class LineupOptimizerAgent(autogen.AssistantAgent):
    def __init__(self, name, data_file, llm_config):
        super().__init__(name=name, llm_config=llm_config)
        self.data = pd.read_csv(data_file)
        self.data['salary'] = self.data['salary'].replace('[\\$,]', '', regex=True).astype(float)
        self.data['projected_points'] = pd.to_numeric(self.data['projected_points'], errors='coerce')
        self.previous_lineups = []
        self.feedback_history = []

    def optimize_lineup(self, constraints: Dict) -> Dict:
        players = self.data.to_dict('records')
        prob = pulp.LpProblem("Fantasy Football", pulp.LpMaximize)
        
        player_vars = pulp.LpVariable.dicts("players", 
                                            ((p['player_name'], p['player_position_id']) for p in players), 
                                            cat='Binary')

        # Objective: Maximize total projected points
        prob += pulp.lpSum([player['projected_points'] * player_vars[player['player_name'], player['player_position_id']] 
                            for player in players])

        # Basic constraints
        self.add_basic_constraints(prob, player_vars, players)
        
        # Apply user-defined constraints
        constraint_status = self.apply_user_constraints(prob, player_vars, players, constraints)
        if not constraint_status['success']:
            return {'error': constraint_status['message']}

        # Apply feedback history
        self.apply_feedback_history(prob, player_vars, players)

        # Solve the problem
        prob.solve()

        if pulp.LpStatus[prob.status] != 'Optimal':
            return {'error': f"Optimization failed: {pulp.LpStatus[prob.status]}"}

        return self.build_lineup(player_vars, players)

    def add_basic_constraints(self, prob, player_vars, players):
        prob += pulp.lpSum([player['salary'] * player_vars[player['player_name'], player['player_position_id']] for player in players]) <= 50000
        prob += pulp.lpSum([player_vars[p['player_name'], 'QB'] for p in players if p['player_position_id'] == 'QB']) == 1
        prob += pulp.lpSum([player_vars[p['player_name'], 'RB'] for p in players if p['player_position_id'] == 'RB']) >= 2
        prob += pulp.lpSum([player_vars[p['player_name'], 'WR'] for p in players if p['player_position_id'] == 'WR']) >= 3
        prob += pulp.lpSum([player_vars[p['player_name'], 'TE'] for p in players if p['player_position_id'] == 'TE']) == 1
        prob += pulp.lpSum([player_vars[p['player_name'], 'DST'] for p in players if p['player_position_id'] == 'DST']) == 1
        prob += pulp.lpSum([player_vars[p['player_name'], p['player_position_id']] for p in players]) == 9

    def apply_user_constraints(self, prob, player_vars, players, constraints):
        if 'must_include' in constraints:
            for player_name in constraints['must_include']:
                matching_players = [p for p in players if p['player_name'].lower().replace(" ", "") == player_name.lower().replace(" ", "")]
                if not matching_players:
                    return {'success': False, 'message': f"Player {player_name} not found in the dataset"}
                prob += pulp.lpSum([player_vars[p['player_name'], p['player_position_id']] for p in matching_players]) == 1

        if 'position_emphasis' in constraints:
            for position, emphasis in constraints['position_emphasis'].items():
                prob += pulp.lpSum([player['projected_points'] * player_vars[p['player_name'], position] 
                                    for p in players if p['player_position_id'] == position]) >= emphasis * prob.objective

        if 'team_preference' in constraints:
            team = constraints['team_preference']
            prob += pulp.lpSum([player_vars[p['player_name'], p['player_position_id']] 
                                for p in players if p['player_team_id'] == team]) >= 3

        return {'success': True, 'message': "Constraints applied successfully"}

    def build_lineup(self, player_vars, players):
        lineup = {
            'QB': None, 'RB1': None, 'RB2': None, 'WR1': None, 'WR2': None, 'WR3': None, 
            'TE': None, 'FLEX': None, 'DST': None
        }

        for player in players:
            if player_vars[player['player_name'], player['player_position_id']].value() == 1:
                position = player['player_position_id']
                if position == 'QB' and lineup['QB'] is None:
                    lineup['QB'] = player
                elif position == 'RB':
                    if lineup['RB1'] is None:
                        lineup['RB1'] = player
                    elif lineup['RB2'] is None:
                        lineup['RB2'] = player
                    elif lineup['FLEX'] is None:
                        lineup['FLEX'] = player
                elif position == 'WR':
                    if lineup['WR1'] is None:
                        lineup['WR1'] = player
                    elif lineup['WR2'] is None:
                        lineup['WR2'] = player
                    elif lineup['WR3'] is None:
                        lineup['WR3'] = player
                    elif lineup['FLEX'] is None:
                        lineup['FLEX'] = player
                elif position == 'TE':
                    if lineup['TE'] is None:
                        lineup['TE'] = player
                    elif lineup['FLEX'] is None:
                        lineup['FLEX'] = player
                elif position == 'DST':
                    lineup['DST'] = player

        self.previous_lineups.append(lineup)
        return lineup

    def reset_lineups(self):
        self.previous_lineups = []

    def format_lineup(self, lineup):
        output = "Optimized Lineup:\n"
        total_salary = 0
        total_points = 0
        for position, player in lineup.items():
            if player:
                output += f"{position}: {player['player_name']} (Projected: {player['projected_points']:.2f}, Salary: ${player['salary']})\n"
                total_salary += player['salary']
                total_points += player['projected_points']
            else:
                output += f"{position}: Not filled\n"
        output += f"\nTotal Salary: ${total_salary}"
        output += f"\nTotal Projected Points: {total_points:.2f}"
        return output

    def apply_feedback_history(self, prob, player_vars, players):
        for feedback in self.feedback_history:
            if feedback['feedback'] == 'good':
                for player in feedback['lineup'].values():
                    if player:
                        prob += pulp.lpSum([player_vars[p['player_name'], p['player_position_id']] 
                                            for p in players if p['player_name'] == player['player_name']]) >= 0.5
            elif feedback['feedback'] == 'bad':
                for player in feedback['lineup'].values():
                    if player:
                        prob += pulp.lpSum([player_vars[p['player_name'], p['player_position_id']] 
                                            for p in players if p['player_name'] == player['player_name']]) <= 0.5

    def add_feedback(self, lineup, feedback):
        self.feedback_history.append({'lineup': lineup, 'feedback': feedback})
        logging.info(f"Added feedback: {feedback} for lineup")

optimizer_agent = LineupOptimizerAgent(
    name="LineupOptimizer",
    data_file='data/merged_fantasy_football_data.csv',
    llm_config=llm_config
)

# User Proxy Agent
user_proxy = autogen.UserProxyAgent(
    name="User",
    human_input_mode="ALWAYS",
    max_consecutive_auto_reply=0
)

def fantasy_football_chat():
    print("Welcome to the Fantasy Football Lineup Generator!")
    print("You can enter specific requests or just press Enter to generate a lineup with default settings.")
    print("Type 'quit' to exit the program.")

    while True:
        try:
            user_input = input("\nEnter your lineup request (or press Enter for default, 'quit' to exit): ").strip()
            if user_input.lower() == 'quit':
                print("Exiting the program. Goodbye!")
                sys.exit(0)

            constraints = {}
            if user_input:
                print("\nUser Input Agent processing request...")
                user_proxy.send(
                    user_input,
                    user_input_agent,
                    request_reply=True
                )
                user_input_response = user_input_agent.last_message()['content']
                print(f"User Input Agent response: {user_input_response}")
                
                try:
                    constraints = json.loads(user_input_response)
                except json.JSONDecodeError:
                    print("Error processing input. Using default settings.")
            else:
                print("\nUsing default settings...")

            num_lineups = constraints.get('num_lineups', 1)
            optimizer_agent.reset_lineups()  # Reset previous lineups

            for i in range(num_lineups):
                print(f"\nGenerating lineup {i+1}...")
                lineup = optimizer_agent.optimize_lineup(constraints)
                
                if 'error' in lineup:
                    print(f"Error generating lineup: {lineup['error']}")
                    continue

                formatted_lineup = optimizer_agent.format_lineup(lineup)
                print(f"\nGenerated Lineup {i+1}:")
                print(formatted_lineup)

                # Get binary feedback
                while True:
                    feedback = input("\nIs this lineup good or bad? (Type 'good' or 'bad'): ").lower()
                    if feedback in ['good', 'bad']:
                        optimizer_agent.add_feedback(lineup, feedback)
                        break
                    else:
                        print("Invalid input. Please type 'good' or 'bad'.")

            print("\n" + "-"*50)
        except KeyboardInterrupt:
            print("\nProgram interrupted. Exiting...")
            sys.exit(0)
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            logging.exception("Unexpected error in fantasy_football_chat")

# Run the chat
if __name__ == "__main__":
    fantasy_football_chat()

Welcome to the Fantasy Football Lineup Generator!
You can enter specific requests or just press Enter to generate a lineup with default settings.
Type 'quit' to exit the program.

User Input Agent processing request...
User (to UserInputAgent):

I want to include josh allen

--------------------------------------------------------------------------------


2024-09-06 03:19:08,973 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 404 Not Found"
2024-09-06 03:19:08,981 - ERROR - Unexpected error in fantasy_football_chat
Traceback (most recent call last):
  File "/tmp/ipykernel_28875/3839370609.py", line 222, in fantasy_football_chat
    user_proxy.send(
  File "/usr/local/lib/python3.11/site-packages/autogen/agentchat/conversable_agent.py", line 732, in send
    recipient.receive(message, self, request_reply, silent)
  File "/usr/local/lib/python3.11/site-packages/autogen/agentchat/conversable_agent.py", line 896, in receive
    reply = self.generate_reply(messages=self.chat_messages[sender], sender=sender)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/autogen/agentchat/conversable_agent.py", line 2050, in generate_reply
    final, reply = reply_func(self, messages=messages, sender=sender, config=reply_func_tuple["config"])


An unexpected error occurred: Error code: 404 - {'error': {'message': 'The model `gpt-4-mini` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}


/usr/local/lib/python3.11/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")



Using default settings...

Generating lineup 1...
Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Sep 26 2023 

command line - /usr/local/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/arm64/cbc /tmp/c29675a55a2c43ada23276b9af153c7a-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/c29675a55a2c43ada23276b9af153c7a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 2452 RHS
At line 2460 BOUNDS
At line 2880 ENDATA
Problem MODEL has 7 rows, 419 columns and 1257 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Error generating lineup: Optimization failed: Infeasible

--------------------------------------------------


/usr/local/lib/python3.11/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")



Using default settings...

Generating lineup 1...
Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Sep 26 2023 

command line - /usr/local/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/arm64/cbc /tmp/9aca3d1b0fb84158a2d15eb8632674ae-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/9aca3d1b0fb84158a2d15eb8632674ae-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 2452 RHS
At line 2460 BOUNDS
At line 2880 ENDATA
Problem MODEL has 7 rows, 419 columns and 1257 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Error generating lineup: Optimization failed: Infeasible

--------------------------------------------------
Exiting the program. Goodbye!


SystemExit: 0

/usr/local/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
